In [7]:
from PLSR_class import PLSRModel
import sys
sys.path.append('../baseline_for_training')
from Dataset import Dataset
import seaborn as sns
import pandas as pd

In [8]:
train_file_name = 'train_data.parquet'
validation_file_name = 'validation_data.parquet'
test_file_name = 'test_data.parquet'


dataset = Dataset(train_file_name,validation_file_name,test_file_name)


In [9]:
is_multi_output = True
PLSR_Tuning = True
param_grid = {'n_components': [i for i in range(1,10)]}
multi_PLSR = PLSRModel(dataset, param_grid, is_multi_output)
single_PLSR = PLSRModel(dataset, param_grid, not is_multi_output, "N_Value")

# Tuning Number of Components

In [11]:
from Training_and_Tuning import hyperParameterTuning

rmses = hyperParameterTuning(multi_PLSR,PLSR_Tuning = PLSR_Tuning)

Optimizing Number of Components: 100%|██████████| 9/9 [00:02<00:00,  4.44it/s]


In [12]:
rmses

{'N_Value': [np.float64(0.47158611168793935),
  np.float64(0.4591919742329582),
  np.float64(0.3933116173519386),
  np.float64(0.32298250894715025),
  np.float64(0.30687633853229956),
  np.float64(0.28207101569704685),
  np.float64(0.20432147438770965),
  np.float64(0.1974107197990201),
  np.float64(0.18333143473633337)],
 'SC_Value': [np.float64(11.781345607541098),
  np.float64(9.699346708521414),
  np.float64(5.868179395888875),
  np.float64(4.263124209623271),
  np.float64(3.6107251699361873),
  np.float64(3.097371367870159),
  np.float64(3.0466958717535566),
  np.float64(2.670162432660288),
  np.float64(2.734118268100288)],
 'ST_Value': [np.float64(50.10546561107106),
  np.float64(45.234164055644534),
  np.float64(34.25139738739402),
  np.float64(24.114697938228574),
  np.float64(20.915324044303873),
  np.float64(19.505548520025176),
  np.float64(17.967997178697345),
  np.float64(17.31151708904473),
  np.float64(17.12653480090013)],
 'Avg_RMSE': [({'copy': True,
    'max_iter': 50

In [13]:
rmses_single = hyperParameterTuning(single_PLSR,PLSR_Tuning = PLSR_Tuning)

Optimizing Number of Components: 100%|██████████| 9/9 [00:00<00:00, 12.44it/s]


In [14]:
rmses_single

{'N_Value': [({'copy': True,
    'max_iter': 500,
    'n_components': 1,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.47158611168793935)),
  ({'copy': True,
    'max_iter': 500,
    'n_components': 2,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.4591919742329582)),
  ({'copy': True,
    'max_iter': 500,
    'n_components': 3,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.3933116173519386)),
  ({'copy': True,
    'max_iter': 500,
    'n_components': 4,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.32298250894715025)),
  ({'copy': True,
    'max_iter': 500,
    'n_components': 5,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.30687633853229956)),
  ({'copy': True,
    'max_iter': 500,
    'n_components': 6,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.28207101569704685)),
  ({'copy': True,
    'max_iter': 500,
    'n_components': 7,
    'scale': True,
    'tol': 1e-06},
   np.float64(0.20432147438770965)),
  ({'copy': True,
    'max_iter'

In [15]:
best_n_components = sorted(rmses['Avg_RMSE'], key = lambda x:x[1])[0][0]['n_components']
print('Best n_components:', best_n_components)

Best n_components: 9


# Training with CV10

In [16]:
from Training_and_Tuning import CV10

multi_PLSR.model.estimator.set_params(n_components = best_n_components)

rmses = CV10(multi_PLSR)

Cross Validation: 100%|██████████| 10/10 [00:03<00:00,  3.03it/s]


In [17]:
rmses

{'N_Value': [np.float64(0.0818098928809392),
  np.float64(0.08492315726309778),
  np.float64(0.1408111177778225),
  np.float64(0.08403872472046955),
  np.float64(0.08536884589278114),
  np.float64(0.08590865882650421),
  np.float64(0.10590480890432503),
  np.float64(0.07517450464698852),
  np.float64(0.08230499825109273),
  np.float64(0.0857496787423151)],
 'SC_Value': [np.float64(1.159493733616769),
  np.float64(1.375695402021692),
  np.float64(1.3976621080250076),
  np.float64(1.497566813958115),
  np.float64(1.4391263428344534),
  np.float64(4.529407505828343),
  np.float64(2.836459210009218),
  np.float64(0.9423900980699347),
  np.float64(1.2209481048741393),
  np.float64(1.5142201468725591)],
 'ST_Value': [np.float64(16.31511578065901),
  np.float64(13.715169914432146),
  np.float64(13.628325704332648),
  np.float64(16.391793986989693),
  np.float64(11.143169400461812),
  np.float64(17.686022812183438),
  np.float64(16.57507963651486),
  np.float64(16.24859334745074),
  np.float64

In [18]:
single_PLSR.model.set_params(n_components = best_n_components)

rmses_single = CV10(single_PLSR)

Cross Validation: 100%|██████████| 10/10 [00:01<00:00,  8.66it/s]


In [19]:
rmses_single

{'N_Value': [np.float64(0.0818098928809392),
  np.float64(0.08492315726309778),
  np.float64(0.1408111177778225),
  np.float64(0.08403872472046955),
  np.float64(0.08536884589278114),
  np.float64(0.08590865882650421),
  np.float64(0.10590480890432503),
  np.float64(0.07517450464698852),
  np.float64(0.08230499825109273),
  np.float64(0.0857496787423151)]}